# Covariates

Covariates are central objects in respy. They can be used to implement complex structures for payoffs, exogenous processes and choices. respy has the standard state space variables `experience`, `period` and `lagged_choice`. These variables can always be used to define payoffs, determine transition probabilities of exogenous processes and restrict choices. Any additional structure has to be imposed using covariates.

In [2]:
# Basic imports
import pandas as pd
import respy as rp

For defining covariates, respy parses python code as strings. This is done in the options dictionary. We will look at an example model here and demonstrate how to define covariates.

In [20]:
params, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)
options

{'solution_draws': 100,
 'solution_seed': 456,
 'n_periods': 5,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'estimation_draws': 100,
 'estimation_seed': 100,
 'estimation_tau': 0.001,
 'interpolation_points': -1,
 'covariates': {'constant': '1'}}

{'solution_draws': 100,
 'solution_seed': 456,
 'n_periods': 5,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'estimation_draws': 100,
 'estimation_seed': 100,
 'estimation_tau': 0.001,
 'interpolation_points': -1,
 'covariates': {'constant': '1'}}

In this very basic model, the only defined covariate is 'constant' which is assigned the constant value of 1. This covariate is then used in params to specify the payoff.

In [5]:
params

value
category       name                       
delta          delta                  0.95
wage_fishing   exp_fishing            0.30
nonpec_fishing constant              -0.20
nonpec_hammock constant               2.00
shocks_sdcorr  sd_fishing             0.50
               sd_hammock             0.50
               corr_hammock_fishing   0.00

The non pecuniary reward for choosing to lie in the hammock is 1 * 2 = 2. So the payoff determined by a covariate is alway the value of this covariate times the return, defined in the value column. From this very simple example, it becomes clear that covariates always need to be numbers or boolean variables, which are then treated as 0 and 1. Let us now define three more complex covariates. Note, that we always append a new covariate to the already existing ones.

In [7]:
# Robinson gets a bonus for fishing, when he was at least three times fishing:
options["covariates"]["at_least_three_times_fishing"] = 'exp_fishing > 2'

In [11]:
# Now we can use this covariate to specify the payoff
params.loc[("wage_fishing", "at_least_three_times_fishing"), "value"] = 0.1

In [13]:
# Robinson gets a bonus for fishing, when he was at least three times fishing and went last period fishing:
options["covariates"][
    "at_least_three_times_fishing_and_last_period"
] = "at_least_three_times_fishing & lagged_choice_1 == fishing"

In [ ]:
# Now we can use this covariate to specify the payoff
params.loc[("wage_fishing", "at_least_three_times_fishing"), "value"] = 0.1

In [9]:

params.sort_values(by="category")

value
category       name                               
delta          delta                          0.95
nonpec_fishing constant                      -0.20
               at_least_three_times_fishing   0.10
nonpec_hammock constant                       2.00
shocks_sdcorr  sd_fishing                     0.50
               sd_hammock                     0.50
               corr_hammock_fishing           0.00
wage_fishing   exp_fishing                    0.30

In [10]:
simulate = rp.get_simulate_func(params, options)
df = simulate(params)

In [9]:
df

Experience_Fishing  Shock_Reward_Fishing  \
Identifier Period                                             
0          0                        0             -0.035035   
           1                        1              0.074254   
           2                        2             -0.354560   
           3                        3             -0.109397   
           4                        4             -1.063705   
...                               ...                   ...   
999        0                        0              0.584099   
           1                        1             -0.391274   
           2                        2              0.394125   
           3                        3              0.531008   
           4                        4              1.367302   

                   Meas_Error_Wage_Fishing  Shock_Reward_Hammock  \
Identifier Period                                                  
0          0                             1              0.040965   
           1                             1              1.506491   
           2                             1              1.185316   
           3                             1             -0.785877   
           4                             1              1.245234   
...                                    ...                   ...   
999        0                             1              1.611990   
           1                             1              0.371305   
           2                             1             -1.448981   
           3                             1             -0.312350   
           4                             1              1.117095   

                   Meas_Error_Wage_Hammock  Dense_Key  Core_Index   Choice  \
Identifier Period                                                            
0          0                             1          0           0  fishing   
           1                             1          1           1  fishing   
           2                             1          2           2  fishing   
           3                             1          3           3  fishing   
           4                             1          4           4  hammock   
...                                    ...        ...         ...      ...   
999        0                             1          0           0  fishing   
           1                             1          1           1  fishing   
           2                             1          2           2  fishing   
           3                             1          3           3  fishing   
           4                             1          4           4  fishing   

                       Wage  Discount_Rate  ...  Nonpecuniary_Reward_Fishing  \
Identifier Period                           ...                                
0          0       0.982635           0.95  ...                         -0.1   
           1       1.400917           0.95  ...                          0.0   
           2       1.526107           0.95  ...                          0.1   
           3       2.328679           0.95  ...                          0.2   
           4            NaN           0.95  ...                          0.3   
...                     ...            ...  ...                          ...   
999        0       1.339169           0.95  ...                         -0.1   
           1       1.110003           0.95  ...                          0.0   
           2       2.219013           0.95  ...                          0.1   
           3       3.207539           0.95  ...                          0.2   
           4       6.577476           0.95  ...                          0.3   

                   Wage_Fishing  Flow_Utility_Fishing  Value_Function_Fishing  \
Identifier Period                                                               
0          0           0.982635              0.882635               10.262244   
           1   

In [28]:
df_with_period

Experience_Fishing  Shock_Reward_Fishing  \
Identifier Period                                             
0          0                        0              1.431303   
           1                        1              0.383519   
           2                        1              0.950278   
           3                        2              0.582585   
           4                        3              1.680125   
...                               ...                   ...   
999        0                        0              1.452422   
           1                        1              1.941229   
           2                        2              1.292309   
           3                        3              0.544626   
           4                        4              0.565707   

                   Meas_Error_Wage_Fishing  Shock_Reward_Hammock  \
Identifier Period                                                  
0          0                             1              0.515252   
           1                             1              0.529793   
           2                             1             -0.189833   
           3                             1             -0.585088   
           4                             1             -0.108781   
...                                    ...                   ...   
999        0                             1             -0.966330   
           1                             1              0.056505   
           2                             1              0.154884   
           3                             1              0.000969   
           4                             1              0.032615   

                   Meas_Error_Wage_Hammock   Choice      Wage  Discount_Rate  \
Identifier Period                                                              
0          0                             1  fishing  1.431303           0.95   
           1                             1  hammock       NaN           0.95   
           2                             1  fishing  1.282740           0.95   
           3                             1  fishing  1.061539           0.95   
           4                             1  fishing  4.132441           0.95   
...                                    ...      ...       ...            ...   
999        0                             1  fishing  1.452422           0.95   
           1                             1  fishing  2.620385           0.95   
           2                             1  fishing  2.354741           0.95   
           3                             1  fishing  1.339565           0.95   
           4                             1  fishing  1.878212           0.95   

                   Present_Bias  Nonpecuniary_Reward_Fishing  Wage_Fishing  \
Identifier Period                                                            
0          0                  1                         -0.2      1.431303   
           1                  1                         -0.1      0.517697   
           2                  1                          0.0      1.282740   
           3                  1                          0.1      1.061539   
           4                  1                          0.2      4.132441   
...                         ...                          ...           ...   
999        0                  1                         -0.2      1.452422   
           1                  1                         -0.1      2.620385   
           2                  1                          0.0      2.354741   
           3                  1                          0.1      1.339565   
           4                  1                          0.2      1.878212   

                   Flow_Utility_Fishing  Value_Function_Fishing  \
Identifier Period                                                 
0          0                   1.231303               10.297683   
           1                   0.417697                8.